In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import streamlit as st
import numpy as np
import psutil

from mainnet_launch.constants import AutopoolConstants

from mainnet_launch.database.schema.full import (
    AutopoolStates,
    Blocks,
    DestinationStates,
    Destinations,
    AutopoolDestinationStates,
    DestinationTokenValues,
    TokenValues,
    Tokens,
)
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
)
from mainnet_launch.data_fetching.get_state_by_block import build_blocks_to_use


def fetch_nav_per_share_and_total_nav(autopool: AutopoolConstants) -> pd.DataFrame:
    nav_per_share_df = merge_tables_as_df(
        [
            TableSelector(
                table=AutopoolStates,
                select_fields=[AutopoolStates.nav_per_share, AutopoolStates.total_nav],
                join_on=None,
                row_filter=(AutopoolStates.autopool_vault_address == autopool.autopool_eth_addr),
            ),
            TableSelector(
                table=Blocks,
                select_fields=Blocks.datetime,
                join_on=(AutopoolStates.chain_id == Blocks.chain_id) & (AutopoolStates.block == Blocks.block),
            ),
        ],
        where_clause=Blocks.block.in_(build_blocks_to_use(autopool.chain)),
        order_by=Blocks.datetime,
    )
    nav_per_share_df = nav_per_share_df.set_index("datetime")
    nav_per_share_df.columns = [autopool.name, autopool.name + "total_nav"]

    nav_per_share_df["30_day_difference"] = nav_per_share_df[autopool.name].diff(periods=30)
    nav_per_share_df["30_day_annualized_return"] = (
        (nav_per_share_df["30_day_difference"] / nav_per_share_df[autopool.name].shift(30)) * (365 / 30) * 100
    )
    nav_per_share_df["7_day_difference"] = nav_per_share_df[autopool.name].diff(periods=7)
    nav_per_share_df["7_day_annualized_return"] = (
        (nav_per_share_df["7_day_difference"] / nav_per_share_df[autopool.name].shift(7)) * (365 / 7) * 100
    )
    nav_per_share_df["daily_return"] = nav_per_share_df[autopool.name].pct_change()
    nav_per_share_df["7_day_MA_return"] = nav_per_share_df["daily_return"].rolling(window=7).mean()
    nav_per_share_df["7_day_MA_annualized_return"] = nav_per_share_df["7_day_MA_return"] * 365 * 100
    nav_per_share_df["30_day_MA_return"] = nav_per_share_df["daily_return"].rolling(window=30).mean()
    nav_per_share_df["30_day_MA_annualized_return"] = nav_per_share_df["30_day_MA_return"] * 365 * 100
    return nav_per_share_df


def fetch_key_metrics_data(autopool: AutopoolConstants):
    nav_per_share_df = fetch_nav_per_share_and_total_nav(autopool)

    destination_state_df = merge_tables_as_df(
        selectors=[
            TableSelector(
                table=AutopoolDestinationStates,
            ),
            TableSelector(
                table=Destinations,
                join_on=(
                    (Destinations.destination_vault_address == AutopoolDestinationStates.destination_vault_address)
                    & (Destinations.chain_id == AutopoolDestinationStates.chain_id)
                ),
                select_fields=[Destinations.pool_type, Destinations.underlying_symbol, Destinations.exchange_name],
            ),
            TableSelector(
                table=DestinationStates,
                select_fields=[
                    DestinationStates.incentive_apr,
                    DestinationStates.fee_apr,
                    DestinationStates.base_apr,
                    DestinationStates.lp_token_safe_price,
                    DestinationStates.total_apr_out,
                    DestinationStates.total_apr_in,
                ],
                join_on=(
                    (AutopoolDestinationStates.destination_vault_address == DestinationStates.destination_vault_address)
                    & (AutopoolDestinationStates.chain_id == DestinationStates.chain_id)
                    & (AutopoolDestinationStates.block == DestinationStates.block)
                ),
            ),
            TableSelector(
                table=Blocks,
                join_on=(
                    (AutopoolDestinationStates.block == Blocks.block)
                    & (AutopoolDestinationStates.chain_id == Blocks.chain_id)
                ),
                select_fields=[Blocks.datetime],
            ),
        ],
        # your global filter (you can also push this into a per‑selector row_filter if you prefer)
        where_clause=(
            (AutopoolDestinationStates.autopool_vault_address == autopool.autopool_eth_addr)
            & (DestinationStates.block.in_(build_blocks_to_use(autopool.chain)))
        ),
        order_by=Blocks.datetime,
        order="asc",
    )

    destination_state_df["unweighted_apr"] = destination_state_df[["fee_apr", "base_apr", "incentive_apr"]].sum(axis=1)

    destination_state_df["safe_tvl_by_destination"] = (
        destination_state_df["lp_token_safe_price"] * destination_state_df["owned_shares"]
    )
    destination_state_df["unweighted_expected_apr"] = 100 * (
        destination_state_df["incentive_apr"] + destination_state_df["base_apr"] + destination_state_df["fee_apr"]
    )

    destination_state_df["readable_name"] = destination_state_df.apply(
        lambda row: f"{row['underlying_symbol']} ({row['exchange_name']})", axis=1
    )

    # this is correct
    safe_tvl_by_destination = (
        destination_state_df.groupby(["datetime", "readable_name"])[["safe_tvl_by_destination"]]
        .sum()
        .reset_index()
        .pivot(values="safe_tvl_by_destination", index="datetime", columns="readable_name")
    )

    total_safe_tvl_over_time = safe_tvl_by_destination.sum(axis=1)
    portion_alloaction_by_destination_df = safe_tvl_by_destination.div(total_safe_tvl_over_time, axis=0)

    max_apr_by_destination = (
        destination_state_df.groupby(["datetime", "readable_name"])[["unweighted_expected_apr"]]
        .max()
        .reset_index()
        .pivot(values="unweighted_expected_apr", index="datetime", columns="readable_name")
    )
    expected_return_series = (max_apr_by_destination * portion_alloaction_by_destination_df).sum(axis=1)

    total_nav_series = nav_per_share_df[autopool.name + "total nav"]
    return (
        nav_per_share_df,
        total_nav_series,
        expected_return_series,
    )


from mainnet_launch.constants import AUTO_ETH, AUTO_USD
import plotly.io as pio

2025-05-16 15:55:24.634 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-05-16 15:55:24.637 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 15:55:24.721 
  command:

    streamlit run /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-16 15:55:24.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 15:55:24.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 15:55:24.722 No runtime found, using MemoryCacheStorageManager
2025-05-16 15:55:25.224 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 15:55:25.225 Thread 'Thread-4

2025-05-16 15:55:25,627 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-16 15:55:25,630 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-16 15:55:25,832 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-16 15:55:25,832 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-16 15:55:26,032 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-16 15:55:26,032 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-16 15:55:26,224 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-16 15:55:26,243 INFO sqlalchemy.engine.Engine SELECT max(blocks.block) AS block 
FROM blocks 
WHERE blocks.chain_id = %(chain_id_1)s AND blocks.block >= %(block_1)s AND blocks.block <= %(block_2)s GROUP BY date_trunc(%(date_trunc_1)s, blocks.datetime) ORDER BY date_trunc(%(date_trunc_2)s, blocks.datetime)
2025-05-16 15:55:26,243 INFO sqlalchemy.engine.Engine [generated in 0.00069s] {'chain_id_1': 1, 'block_1': 20752910, 'block_2': 100000000, 'date_trunc_1': 'day', 'dat

2025-05-16 15:55:26.525 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-16 15:55:26.527 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-05-16 15:55:26,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-16 15:55:26,624 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-16 15:55:26,624 INFO sqlalchemy.engine.Engine [generated in 0.00144s] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x11e992b00>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-16 15:55:26,626 INFO sqlalchemy.engine.Engine SELECT
    autopool_destination_states.*,
    destinations.pool_type,
    destinations.underlying_symbol,
    destinations.exchan

,destination_vault_address,autopool_vault_address,block,chain_id,owned_shares,pool_type,underlying_symbol,exchange_name,incentive_apr,fee_apr,base_apr,lp_token_safe_price,total_apr_out,total_apr_in,datetime
0,0x49895f72fd9d0BF6BBb485C70CE38556de62b070,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20759464,1,0.000000,curveNG,osETH-rETH,curve,0.000000,0.028268,0.026397,1.026872,0.052995,0.052995,2024-09-15 23:59:59+00:00
1,0xaD70ed6A904d38acD0BbE2D922279149cFEDb190,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20759464,1,0.000000,curveV1,frxeth-ng-f,curve,0.027456,0.002013,0.000000,1.001118,0.027233,0.026724,2024-09-15 23:59:59+00:00
2,0xdfE3fA7027E84f59b266459C567278C79fe86f0C,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20759464,1,NaN,balCompStable,ETHx/wstETH,balancer,NaN,NaN,NaN,1.010355,NaN,NaN,2024-09-15 23:59:59+00:00
3,0xB84c580c005A5F633C8CF6eecAf5b0a8314EC5c4,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20759464,1,0.000000,curveNG,pxsteth,curve,0.083241,0.000371,0.012370,1.008197,0.088808,0.087657,2024-09-15 23:59:59+00:00
4,0x84083bdcbF44960f0745dd0b794F426568D762b2,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,20759464,1,0.000000,curveNG,pxethweth,curve,0.114819,0.001478,0.000000,0.999302,0.105940,0.104815,2024-09-15 23:59:59+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10930,0x6a8C6ff78082a2ae494EB9291DdC7254117298Ff,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22491938,1,0.000000,curveNG,wETHrETH,curve,0.037057,0.017865,0.013955,1.023675,0.065523,0.065171,2025-05-15 23:59:59+00:00
10931,0x2899143298FECa3a36DAfDDcddAF167f1D673254,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22491938,1,0.000000,balCompStable,ETHx/wstETH,balancer,0.096677,0.007136,0.014112,1.025327,0.108854,0.108257,2025-05-15 23:59:59+00:00
10932,0x2B08137BeABd2454AD3631DEB754F97C5c93eB78,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22491938,1,0.000000,curveNG,wETHrETH,curve,0.037057,0.017865,0.013955,1.023675,0.065523,0.065171,2025-05-15 23:59:59+00:00
10933,0x1Ea622fa030e4a78F4CC2f305dd3c08DA3F08573,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22491938,1,1650.286018,curveV1,ethx-f,curve,0.052725,0.019213,0.016622,1.040758,0.083680,0.083288,2025-05-15 23:59:59+00:00


In [ ]:
# this is correct
safe_tvl_by_destination = destination_state_df.pivot(
    values="safe_tvl_by_destination", index="datetime", columns="destination_vault_address"
)
total_safe_tvl_over_time = safe_tvl_by_destination.sum(axis=1)
portion_alloaction_by_destination_df = safe_tvl_by_destination.div(total_safe_tvl_over_time, axis=0)

expected_apr_by_destination = destination_state_df.pivot(
    values="unweighted_expected_apr", index="datetime", columns="destination_vault_address"
)

expected_apr = (expected_apr_by_destination * portion_alloaction_by_destination_df).sum(axis=1)


# px.bar(total_safe_tvl_over_time).show()
# px.bar(safe_tvl_by_destination).show()
# px.bar(portion_alloaction_by_destination_df).show()
px.line(expected_apr)

In [ ]:
destination_state_df.sort_values("total_apr_out", ascending=False).head()[
    ["block", "destination_vault_address"]
].to_csv()

In [ ]:
# this is correct
safe_tvl_by_destination = (
    destination_state_df.groupby(["datetime", "readable_name"])[["safe_tvl_by_destination"]]
    .sum()
    .reset_index()
    .pivot(values="safe_tvl_by_destination", index="datetime", columns="readable_name")
)

total_safe_tvl_over_time = safe_tvl_by_destination.sum(axis=1)
portion_alloaction_by_destination_df = safe_tvl_by_destination.div(total_safe_tvl_over_time, axis=0)

min_apr_by_destination = (
    destination_state_df.groupby(["datetime", "readable_name"])[["unweighted_expected_apr"]]
    .min()
    .reset_index()
    .pivot(values="unweighted_expected_apr", index="datetime", columns="readable_name")
)
max_apr_by_destination = (
    destination_state_df.groupby(["datetime", "readable_name"])[["unweighted_expected_apr"]]
    .max()
    .reset_index()
    .pivot(values="unweighted_expected_apr", index="datetime", columns="readable_name")
)
expected_apr = (max_apr_by_destination * portion_alloaction_by_destination_df).sum(axis=1)
px.line((max_apr_by_destination * portion_alloaction_by_destination_df).sum(axis=1))
# expected APR, this is right

# expected_apr
# # px.bar(total_safe_tvl_over_time).show()
# px.bar(safe_tvl_by_destination).show()
# px.bar(portion_alloaction_by_destination_df).show()
# px.line(expected_apr_by_destination)

In [ ]:
destination_state_df.sort_values("unweighted_expected_apr").dropna()

In [ ]:
expected_apr_by_destination = 100 * destination_state_df.groupby(["underlying_symbol", "datetime"])[
    ["total_apr_out"]
].max().reset_index().pivot(values="total_apr_out", index="datetime", columns="underlying_symbol")

px.line(expected_apr_by_destination)

In [ ]:
destination_state_df["underlying_symbol"].value_counts()

In [ ]:
expected_apr_by_destination = 100 * destination_state_df.groupby(["underlying_symbol", "datetime"])[
    ["total_apr_in"]
].first().reset_index().pivot(values="total_apr_in", index="datetime", columns="underlying_symbol")

px.line(expected_apr_by_destination)

In [ ]:
px.line(destination_state_df[destination_state_df["underlying_symbol"] == "weeth-ng"]["total_apr_out"])

In [ ]:
destination_state_df[destination_state_df["underlying_symbol"] == "weeth-ng"].sort_values(
    "total_apr_out", ascending=False
)

In [ ]:
100 * 23791992517125747 / 1e18

In [ ]:
# at block 21339732 ( 2024-12-05 23:59:59+00:00 )
# 0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126
# it is correct, but we din't have an allocation ther at the time
# get destination _summary_stats for  0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56 autoETH says incentive APR is 21%

In [ ]:
from multicall import Call
from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    safe_normalize_with_bool_success,
)
from mainnet_launch.constants import ETH_CHAIN
from mainnet_launch.data_fetching.get_state_by_block import build_blocks_to_use

stats = "0xF25b36c77e869b01477860a0EABc4f4Ff745EAC3"
apr_call = Call(
    "0xF25b36c77e869b01477860a0EABc4f4Ff745EAC3",
    ["lastSnapshotTotalAPR()(uint256)"],
    # the tuple key here will become the column name in your df
    [("lastSnapshotTotalAPR", safe_normalize_with_bool_success)],
)

# 3) Get the blocks you want to sample
blocks = build_blocks_to_use(ETH_CHAIN)

df_apr = get_raw_state_by_blocks(
    calls=[apr_call],
    blocks=blocks,
    chain=ETH_CHAIN,
    include_block_number=True,
)


px.line(df_apr)

In [ ]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_states_table import *
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_states_table import (
    _build_summary_stats_call,
)

autopools_orm: list[Autopools] = get_full_table_as_orm(Autopools, where_clause=Autopools.chain_id == ETH_CHAIN.chain_id)
autopools_orm

In [ ]:
call = _build_summary_stats_call(autopools_orm[2], "0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126", "in")

stat_df = get_raw_state_by_blocks(
    calls=[call],
    blocks=blocks,
    chain=ETH_CHAIN,
    include_block_number=True,
)

In [ ]:
import pandas as pd

# pick out the column you care about
col = ("0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56", "0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126", "in")

# collect into a normal Python list
records = []
for r in stat_df[col].values:
    if r is None:
        # replace None with an empty dict
        records.append({})
    else:
        records.append(r)

# now build your DataFrame
weeNG_stat_df = pd.DataFrame.from_records(records)
weeNG_stat_df.index = stat_df.index

px.line(weeNG_stat_df["compositeReturn"])

In [ ]:
weeNG_stat_df.columns

In [ ]:
px.bar(weeNG_stat_df["ownedShares"])

### Something is up with weeth-ng

In [ ]:
# destination_state_df = merge_tables_as_df(
#     selectors=[
#         TableSelector(
#             DestinationStates,
#             select_fields=[
#                 DestinationStates.block,
#                 DestinationStates.price_per_share,
#                 DestinationStates.price_return,
#                 DestinationStates.underlying_token_total_supply,
#             ],
#         ),
#         TableSelector(
#             DestinationTokenValues,
#             select_fields=[
#                 DestinationTokenValues.spot_price,
#                 DestinationTokenValues.quantity,
#             ],
#             join_on=(
#                 (DestinationStates.destination_vault_address == DestinationTokenValues.destination_vault_address)
#                 & (DestinationStates.chain_id == DestinationTokenValues.chain_id)
#                 & (DestinationStates.block == DestinationTokenValues.block)
#             ),
#         ),
#         TableSelector(
#             Destinations,
#             select_fields=[
#                 Destinations.underlying_symbol,
#                 Destinations.pool_type,
#             ],
#             join_on=(
#                 (Destinations.destination_vault_address == DestinationStates.destination_vault_address)
#                 & (Destinations.chain_id == DestinationStates.chain_id)
#             ),
#         ),
#         TableSelector(
#             AutopoolDestinationStates,
#             select_fields=[AutopoolDestinationStates.owned_shares],
#             join_on=(
#                 (DestinationStates.destination_vault_address == AutopoolDestinationStates.destination_vault_address)
#                 & (DestinationStates.chain_id == AutopoolDestinationStates.chain_id)
#                 & (DestinationStates.block == AutopoolDestinationStates.block)
#             ),
#         ),
#         TableSelector(
#             TokenValues,
#             select_fields=[TokenValues.safe_price, TokenValues.backing, TokenValues.denominated_in],
#             join_on=(
#                 (DestinationTokenValues.token_address == TokenValues.token_address)
#                 & (DestinationTokenValues.chain_id == TokenValues.chain_id)
#                 & (DestinationTokenValues.block == TokenValues.block)
#             ),
#             row_filter=(TokenValues.denominated_in == AUTO_USD.base_asset),
#         ),
#         TableSelector(
#             Tokens,
#             select_fields=[Tokens.symbol,],
#             join_on=(
#                 (Tokens.token_address == TokenValues.token_address)
#                 & (Tokens.chain_id == TokenValues.chain_id)
#             ),
#             row_filter=(TokenValues.denominated_in == AUTO_USD.base_asset),
#         ),
#         TableSelector(
#             Blocks,
#             select_fields=[Blocks.datetime],
#             join_on=((DestinationStates.chain_id == Blocks.chain_id) & (DestinationStates.block == Blocks.block)),
#         ),
#     ],
#     where_clause=(
#         (AutopoolDestinationStates.autopool_vault_address == AUTO_USD.autopool_eth_addr)
#         & (Blocks.block.in_(build_blocks_to_use(AUTO_USD.chain)))
#     ),
#     order_by=Blocks.datetime,
# )

# destination_state_df

In [ ]:
destination_state_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            DestinationStates,
            select_fields=[
                DestinationStates.price_return,
                DestinationStates.underlying_token_total_supply,
            ],
        ),
        TableSelector(
            Destinations,
            select_fields=[
                Destinations.underlying_symbol,
                Destinations.pool_type,
            ],
            join_on=(
                (Destinations.destination_vault_address == DestinationStates.destination_vault_address)
                & (Destinations.chain_id == DestinationStates.chain_id)
            ),
        ),
        TableSelector(
            AutopoolDestinationStates,
            select_fields=[AutopoolDestinationStates.owned_shares],
            join_on=(
                (DestinationStates.destination_vault_address == AutopoolDestinationStates.destination_vault_address)
                & (DestinationStates.chain_id == AutopoolDestinationStates.chain_id)
                & (DestinationStates.block == AutopoolDestinationStates.block)
            ),
        ),
        TableSelector(
            Blocks,
            select_fields=[Blocks.datetime],
            join_on=((DestinationStates.chain_id == Blocks.chain_id) & (DestinationStates.block == Blocks.block)),
        ),
    ],
    where_clause=(
        (AutopoolDestinationStates.autopool_vault_address == AUTO_ETH.autopool_eth_addr)
        & (Blocks.block.in_(build_blocks_to_use(AUTO_ETH.chain)))
    ),
    order_by=Blocks.datetime,
)

destination_state_df

In [ ]:
destination_state_df["portion_ownership"] = (
    destination_state_df["owned_shares"] / destination_state_df["underlying_token_total_supply"]
)
destination_state_df

In [ ]:
total_supply_df = (
    destination_state_df.groupby(["datetime", "underlying_symbol"])["underlying_token_total_supply"]
    .sum()
    .reset_index()
    .pivot(index="datetime", values="underlying_token_total_supply", columns="underlying_symbol")
    .fillna(0)
)


owned_shares_df = (
    destination_state_df.groupby(["datetime", "underlying_symbol"])["owned_shares"]
    .sum()
    .reset_index()
    .pivot(index="datetime", values="owned_shares", columns="underlying_symbol")
    .fillna(0)
)

In [ ]:
portion_ownership_df = owned_shares_df / total_supply_df

px.line(portion_ownership_df)

In [ ]:
price_return_df = (
    destination_state_df.groupby(["datetime", "underlying_symbol"])["price_return"]
    .min()
    .reset_index()
    .pivot(index="datetime", values="price_return", columns="underlying_symbol")
    .fillna(0)
)

px.line(price_return_df)

In [ ]:
px.line((-100 * price_return_df * portion_ownership_df).sum(axis=1))
# the portion is wrong, it should be out TVL / autopool tvl
# not out % ownerhsip of the pool

In [ ]:
destination_state_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            DestinationStates,
            select_fields=[
                DestinationStates.destination_vault_address,
                DestinationStates.block,
                DestinationStates.incentive_apr,
                DestinationStates.fee_apr,
                DestinationStates.base_apr,
                DestinationStates.price_per_share,
                DestinationStates.price_return,
                DestinationStates.lp_token_spot_price,
            ],
        ),
        TableSelector(
            AutopoolDestinationStates,
            [
                AutopoolDestinationStates.owned_shares,
            ],
            (DestinationStates.destination_vault_address == AutopoolDestinationStates.destination_vault_address)
            & (DestinationStates.chain_id == AutopoolDestinationStates.chain_id)
            & (DestinationStates.block == AutopoolDestinationStates.block),
        ),
        TableSelector(
            Destinations,
            [Destinations.pool_type],
            (DestinationStates.destination_vault_address == Destinations.destination_vault_address)
            & (DestinationStates.chain_id == Destinations.chain_id),
        ),
        TableSelector(
            AutopoolStates,
            [AutopoolStates.total_nav],
            (AutopoolStates.autopool_vault_address == AUTO_ETH.autopool_eth_addr)
            & (AutopoolStates.chain_id == DestinationStates.chain_id)
            & (AutopoolStates.block == DestinationStates.block),
        ),
        TableSelector(
            Blocks,
            Blocks.datetime,
            (DestinationStates.block == Blocks.block) & (DestinationStates.chain_id == Blocks.chain_id),
        ),
    ],
    where_clause=(AutopoolDestinationStates.autopool_vault_address == AUTO_ETH.autopool_eth_addr)
    & (Blocks.block.in_(build_blocks_to_use(AUTO_ETH.chain))),
    order_by=Blocks.datetime,
)

owned_shares_df = destination_state_df.pivot(
    index="datetime", values="owned_shares", columns="destination_vault_address"
)

price_per_share_df = destination_state_df.pivot(
    index="datetime", values="price_per_share", columns="destination_vault_address"
)

allocation_df = (price_per_share_df * owned_shares_df).fillna(0)
total_nav_series = allocation_df.sum(axis=1)

portion_df = allocation_df.div(total_nav_series, axis=1)

destination_state_df["unweighted_apr"] = destination_state_df[["fee_apr", "base_apr", "incentive_apr"]].sum(axis=1)

uwcr_df = destination_state_df.pivot(index="datetime", values="unweighted_apr", columns="destination_vault_address")
expected_return_series = 100 * (portion_df.fillna(0) * uwcr_df.fillna(0)).sum(axis=1)

price_return_df = destination_state_df.pivot(
    index="datetime", values="price_return", columns="destination_vault_address"
)

# pretty sure the issue here is that it is not properly grouping values by price return
# price return is still not correct
weighted_price_return_series = -100 * (portion_df.fillna(0) * price_return_df.fillna(0)).sum(axis=1)

In [ ]:
owned_shares_df = destination_state_df.pivot(
    index="datetime", values="owned_shares", columns="destination_vault_address"
).fillna(0)
owned_shares_df

In [ ]:
price_per_share_df = destination_state_df.pivot(
    index="datetime", values="price_per_share", columns="destination_vault_address"
)
price_per_share_df

In [ ]:
destination_state_df = merge_tables_as_df(
    selectors=[
        # 1) Base table: DestinationStates
        TableSelector(
            DestinationStates,
            select_fields=[
                DestinationStates.destination_vault_address,
                DestinationStates.underlying_token_total_supply,
            ],
        ),
        # 2) AutopoolDestinationStates → join on vault+chain+block
        TableSelector(
            AutopoolDestinationStates,
            select_fields=[AutopoolDestinationStates.owned_shares],
            join_on=(
                (DestinationStates.destination_vault_address == AutopoolDestinationStates.destination_vault_address)
                & (DestinationStates.chain_id == AutopoolDestinationStates.chain_id)
                & (DestinationStates.block == AutopoolDestinationStates.block)
            ),
        ),
        # 4) Blocks → join on block+chain
        TableSelector(
            Blocks,
            select_fields=[Blocks.datetime],
            join_on=((DestinationStates.block == Blocks.block) & (DestinationStates.chain_id == Blocks.chain_id)),
        ),
    ],
    where_clause=(
        (AutopoolDestinationStates.autopool_vault_address == AUTO_ETH.autopool_eth_addr)
        & (Blocks.block.in_(build_blocks_to_use(AUTO_ETH.chain)))
    ),
    order_by=Blocks.datetime,
)

# care about (destination_vault_address,
destination_state_df["portion_ownership"] = (
    destination_state_df["owned_shares"] / destination_state_df["underlying_token_total_supply"]
)

In [ ]:
destination_state_df[["datetime", "destination_vault_address"]].value_counts().sort_values()

In [ ]:
token_value_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            DestinationTokenValues,
            select_fields=[
                DestinationTokenValues.destination_vault_address,
                DestinationTokenValues.token_address,
                DestinationTokenValues.quantity,
            ],
        ),
        TableSelector(
            TokenValues,
            select_fields=[TokenValues.safe_price, TokenValues.backing],
            join_on=(
                (TokenValues.chain_id == DestinationTokenValues.chain_id)
                & (TokenValues.block == DestinationTokenValues.block)
                & (TokenValues.token_address == DestinationTokenValues.token_address)
                & (TokenValues.denominated_in == AUTO_ETH.base_asset)
            ),
        ),
        TableSelector(
            Tokens,
            select_fields=[Tokens.symbol],
            join_on=((Tokens.chain_id == TokenValues.chain_id) & (Tokens.token_address == TokenValues.token_address)),
        ),
        TableSelector(
            Blocks,
            select_fields=[Blocks.datetime],
            join_on=((TokenValues.block == Blocks.block) & (TokenValues.chain_id == Blocks.chain_id)),
        ),
    ],
    where_clause=((Blocks.block.in_(build_blocks_to_use(AUTO_ETH.chain)))),
    order_by=Blocks.datetime,
)

token_value_df

In [ ]:
df = pd.merge(
    token_value_df,
    destination_state_df,
    on=["datetime", "destination_vault_address"],
    how="left",
)
df["safe_value"] = df["quantity"] * df["safe_price"] * df["portion_ownership"]
df["backing_value"] = df["quantity"] * df["backing"] * df["portion_ownership"]

total_safe_and_backing_value = df.groupby(["datetime", "symbol"])[["safe_value", "backing_value"]].sum().reset_index()

safe_value_df = total_safe_and_backing_value.pivot(index="datetime", values="safe_value", columns="symbol").fillna(0)
backing_value_df = total_safe_and_backing_value.pivot(
    index="datetime", values="backing_value", columns="symbol"
).fillna(0)

px.bar(safe_value_df)

In [ ]:
px.line((backing_value_df - safe_value_df))

In [ ]:
df["price_return"] = 100 * (df["backing"] - df["safe_price"]) / df["backing"]

price_return_df = df.groupby(["datetime", "symbol"])["price_return"].first().reset_index()
price_return_df

In [ ]:
px.line(100 * (backing_value_df.sum(axis=1) - safe_value_df.sum(axis=1)) / backing_value_df.sum(axis=1))